## Advanced Computing Concepts with Dynamic Compute
In other tutorials we explored the fundementals of `Dynamic Compute`. In this notebook we will provide an overview of a subset of more complex examples of the API, such as:
* Interacting between `Mosaic`s and `ImageStack`s
* Interoperability between `DLTile` and other `AOI` objects
* More advanced batch-style, time series analysis examples

In [ ]:
import descarteslabs as dl
import descarteslabs.dynamic_compute as dc
from descarteslabs.dynamic_compute import Mosaic, ImageStack

First, we will set up some global variables and set up our input `ImageStack` and `Mosaic` objects. Here we will work with an `ImageStack` of `Sentinel-2` imagery corresponding to the date range of January to May, 2023 and a `Mosaic` of 2022's `Cropland Data Layer`:

In [ ]:
start_date = "2023-01-01"
end_date = "2023-05-01"
pid = "esa:sentinel-2:l2a:v1"
cdl_pid = "usda:cdl:v1"
bands = "nir red green blue"

In [ ]:
s2_stack = ImageStack.from_product_bands(
    pid, bands, start_datetime=start_date, end_datetime=end_date
).filter(lambda x: x.cloud_fraction < 0.2)

cdl_mosaic = Mosaic.from_product_bands(
    cdl_pid, "class", start_datetime="2022-01-01", end_datetime="2023-01-01"
)

### Interacting between ImageCollecitons and Mosaics
In the next few cells we will calculate NDVI through our time period and then mask to our Cropland Data Layer's Corn class:

In [ ]:
nir, red = s2_stack.unpack_bands("nir red")
ndvi = (nir - red) / (nir + red)

In [ ]:
ndvi_corn_mask = ndvi.mask(cdl_mosaic != 1)

In [ ]:
m = dc.map
m.center = 41.34232959809853, -95.54491138405865
m.zoom = 14

In [ ]:
_ = (
    s2_stack.pick_bands("red green blue")
    .median(axis="images")
    .visualize("Sentinel-2 Composite", m)
)
_ = ndvi.median(axis="images").visualize("NDVI Composite", m)
_ = ndvi_corn_mask.median(axis="images").visualize(
    "NDVI Composite Corn Mask", m, colormap="magma"
)
_ = cdl_mosaic.pick_bands("class").visualize("CDL", m, colormap="terrain")

In [ ]:
m

### Interoperability with other `GeoContext` Objects

In [ ]:
from shapely.wkt import loads

You can create and `compute` over an `AOI` generated from a `shapely Polygon`:

In [ ]:
geom = loads(
    """POLYGON ((-95.54491138405865 41.34232959809853, 
    -95.52455234632363 41.34232959809853, 
    -95.52455234632363 41.35521625255075, 
    -95.54491138405865 41.35521625255075, 
    -95.54491138405865 41.34232959809853))"""
)
aoi = dl.geo.AOI(geom, resolution=10.0, crs="EPSG:3857")

In [ ]:
ndvi_mask_arr = ndvi_corn_mask.median(axis="images").compute(aoi).ndarray
dl.utils.display(ndvi_mask_arr[0, :, :], colormap="viridis")

And also over `DLTile` objects:

In [ ]:
dltile = dl.geo.DLTile.from_latlon(
    41.34232959809853, -95.54491138405865, tilesize=512, pad=0, resolution=10.0
)

In [ ]:
ndvi_mask_arr = ndvi_corn_mask.median(axis="images").compute(dltile).ndarray
dl.utils.display(ndvi_mask_arr, colormap="viridis")

### Batch Style Compute - Time Series Analysis
In the following cells we will demonstrate how you can utilize an `ImageStack` to retrieve time series summary statistics and `ndarrays`. First, we'll call `.compute` over our `ImageStack` without an aggregation method (e.g. `median`):

In [ ]:
ndvi_corn_mask_stack = ndvi_corn_mask.compute(dltile)
type(ndvi_corn_mask_stack)

In [ ]:
ndvi_corn_mask_stack.ndarray.shape

Next we will compute the corresponding `mean` NDVI value for each `Image` within our `ImageStack`:

In [ ]:
ndvi_corn_mask_mean = ndvi_corn_mask.mean(axis="pixels").compute(dltile)
ndvi_corn_mask_mean_list = ndvi_corn_mask_mean.ndarray[:, 0].tolist()

In [ ]:
ndvi_corn_mask_mean_list

In [ ]:
ndvi_mask_dates = [
    p["acquired"].strftime("%Y-%m-%d %HH:%mm:%s")
    for p in ndvi_corn_mask_stack.properties
]
ndvi_mask_dates

In [ ]:
ndvi_mask_ids = [p["id"] for p in ndvi_corn_mask_stack.properties]
ndvi_mask_ids

In [ ]:
title_list = []
for i in range(len(ndvi_mask_dates)):
    title_list.append(
        f"Mean NDVI: {ndvi_corn_mask_mean_list[i]}\nAcquired Date: {ndvi_mask_dates[i]}\nImage ID: {ndvi_mask_ids[0]}"
    )

In [ ]:
dl.utils.display(*ndvi_corn_mask_stack.ndarray, title=title_list, colormap="viridis")